# 載入所需函式

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
import os

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPool2D
from tensorflow.python.keras.utils import np_utils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 載入水果分類所在的各個資料夾

# 訓練資料

In [3]:
path = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/project/photos/train'
files = os.listdir(path)
dirs = list()

In [4]:
# 將目錄中的各個瓜類資料夾抓到列表中
for dir in files:
  if os.path.isdir(os.path.join(path, dir)):
    dirs.append(dir)
dirs

['絲瓜', '苦瓜', '哈密瓜', '木瓜', '冬瓜', '瓠瓜', '小黃瓜', '櫛瓜', '南瓜', '西瓜']

In [5]:
# 自訂將圖片縮小的函式
def resizeImg(dir_path, imgs):
  size = (250,250)
  for item in imgs:
    item = os.path.join(dir_path, item)
    image = cv2.imread(item)
    h, w = image.shape[:2]
    # 找最大邊：如高或寬大於100
    if h > size[0] or w > size[1]:
      # 等比例縮小
      ratio = max(h/size[0], w/size[1]) # 找最大比例
      image = cv2.resize(image, (int(w/ratio), int(h/ratio)))
    # 重新抓取新的高、寬
    h, w = image.shape[:2]
    # 高度的黑邊
    pad_h = size[0]-h
    # 寬度的黑邊
    pad_w = size[1]-h
    # 上、下補黑邊的高度
    top, bottom = pad_h//2, pad_h-(pad_h//2)
    # 左、右補黑邊的寬度
    left, right = pad_w//2, pad_w-(pad_w//2)
    # 補黑邊
    image = cv2.copyMakeBorder(image, top, bottom, left, right, cv2.BORDER_CONSTANT, value=[0,0,0])
    # 抓檔名
    name = item.split('/')[-1]
    # 將改好的檔案寫進新的資料夾中
    cv2.imwrite(f"{dir_path}/resize/{name}", image)

In [6]:
x_train = list()
y_train = list()

def createData(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 訓練題目
    x_train.append(img_array)
    # 訓練答案
    y_train.append(category)

In [7]:
category = 0
for directory in dirs:
  dir_path = os.path.join(path, directory)
  # 目錄中的所有檔案
  dir_file = os.listdir(dir_path)
  img_files = list()
  for file in dir_file:
    # 如附檔名為這三種的任何一種
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
      img_files.append(file)
  resizeDir = os.path.join(dir_path, 'resize')
  if not os.path.exists(resizeDir):
    os.makedirs(resizeDir)
    resizeImg(dir_path, img_files)
  # 分類
  createData(resizeDir, category)
  category += 1

In [7]:
len(y_train)

0

# 測試資料

In [8]:
test_path = '/content/drive/MyDrive/2023聯成電腦/Python IV 視覺辨識與機器學習/project/photos/test'
test_files = os.listdir(test_path)
test_dirs = list()

In [9]:
for dir in test_files:
  if os.path.isdir(os.path.join(test_path, dir)):
    test_dirs.append(dir)

In [10]:
x_test = list()
y_test = list()

def createTestData(path, category):
  for p in os.listdir(path):
    # 轉灰階
    img_array = cv2.imread(os.path.join(path, p), 0)
    # 訓練題目
    x_test.append(img_array)
    # 訓練答案
    y_test.append(category)

In [11]:
test_category = 0
for directory in test_dirs:
  dir_path = os.path.join(test_path, directory)
  # 目錄中的所有檔案
  dir_file = os.listdir(dir_path)
  img_files = list()
  for file in dir_file:
    # 如附檔名為這三種的任何一種
    if file.endswith('.jpg') or file.endswith('.png') or file.endswith('.jpeg'):
      img_files.append(file)
  resizeDir = os.path.join(dir_path, 'resize')
  if not os.path.exists(resizeDir):
    os.makedirs(resizeDir)
    resizeImg(dir_path, img_files)
  # 分類
  createTestData(resizeDir, test_category)
  test_category += 1

In [12]:
len(x_test)

294

In [13]:
len(x_train)

0

# 打散成組的資料

In [16]:
# reshape(-1,250,250,1) 一組資料一維改成一個資料一維
x_train = np.array(x_train).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_train = np.array(y_train)

x_train = x_train / 255 # 色階 => 收斂
# 將答案分類 OneHot
y_train = np_utils.to_categorical(y_train)

x_test = np.array(x_test).reshape(-1,250,250,1) # (張數, 長, 寬, 通道數)
y_test = np.array(y_test)

x_test = x_test / 255 # 色階 => 收斂
y_test = np_utils.to_categorical(y_test)

ValueError: cannot reshape array of size 1094 into shape (250,250,1)

# 打亂資料群

In [ ]:
from sklearn.model_selection import train_test_split
# 訓練
rX_train, rX_test, ry_train, ry_test = train_test_split(x_train, y_train, test_size=1, random_state=1)
# 測試
raX_train, raX_test, ray_train, ray_test = train_test_split(x_test, y_test, test_size=1, random_state=1)

# 建模

In [ ]:
model = Sequential()

In [ ]:
# 卷積
model.add(Conv2D(filters=32, kernel_size=(3,3), padding='same', input_shape=x_train.shape[1:], activation='relu'))
# 池化
model.add(MaxPool2D(pool_size=(2,2))) # pool_size=(2,2) 為預設，不寫也可以
# 卷積
model.add(Conv2D(filters=64, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
model.add(MaxPool2D(pool_size=(2,2)))
# 卷積
model.add(Conv2D(filters=128, kernel_size=(3,3), padding='same', activation='relu'))
# 池化
model.add(MaxPool2D(pool_size=(2,2)))
# 放棄的比率
model.add(Dropout(0.3))
# 扁平化
model.add(Flatten())
# 放棄的比率
model.add(Dropout(0.2))
# 連接層：通常用1024(三通道 32*32*32)
model.add(Dense(1024, activation='relu'))
# 輸出層：10種分類
model.add(Dense(10, activation='softmax'))

# 編譯(程式)

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 訓練

In [ ]:
model.fit(rX_train, ry_train, epochs=15, batch_size=30, validation_data=(x_test, y_test))

# 測試

In [ ]:
predict1 = model.predict(rX_train)

In [ ]:
predict2 = model.predict(raX_train)

In [ ]:
right = 0
for i in range(len(predict1)):
  ans = np.argmax(predict1[i])
  pre = ry_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict1)

In [ ]:
right = 0
for i in range(len(predict2)):
  ans = np.argmax(predict2[i])
  pre = ray_train[i][ans]
  if pre == 1:
    right += 1
right / len(predict2)